#### Pre-processing e class balancing

In [1]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 1. Lettura dei dati
X_train = pd.read_csv("Dataset/Train/X_train.csv")
y_train = pd.read_csv("Dataset/Train/y_train.csv")

# 2. Unire X_train e y_train per mantenere le etichette
df_train = pd.merge(X_train, y_train, on="id")

# Separare le feature e le etichette
X = df_train.drop(columns=["id", "faulty", "trq_margin"])  # Rimuoviamo id, faulty, trq_margin
y = df_train[["faulty", "trq_margin"]]  # Manteniamo "faulty" e "trq_margin"

target_size = 400000

undersampler = RandomUnderSampler(sampling_strategy=1.0, random_state=42)  # Undersampling della classe maggioritaria
oversampler = SMOTE(sampling_strategy=1.0, random_state=42)  # Oversampling della classe minoritaria

# Applicare l'undersampling
X_undersampled, y_undersampled = undersampler.fit_resample(X, y["faulty"])

# Se il numero di campioni è inferiore al target_size, applicare SMOTE per raggiungere 400.000 campioni
if len(X_undersampled) < target_size:
    X_resampled, y_resampled = oversampler.fit_resample(X_undersampled, y_undersampled)
else:
    X_resampled, y_resampled = X_undersampled, y_undersampled

# Se i dati finali sono superiori a 400.000, riduciamo a 400.000
if len(X_resampled) > target_size:
    X_resampled, _, y_resampled, _ = train_test_split(X_resampled, y_resampled, train_size=target_size, random_state=42)

# 4. Standardizzare i dati
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_resampled)

# 5. Creare i nuovi DataFrame per X_train e y_train
df_X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Convertire y_resampled in un formato 1D (array semplice)
y_resampled = y_resampled.values.flatten()

# Creare un DataFrame per y_train, che include anche trq_margin
y_train_final = pd.DataFrame({
    'id': df_train['id'][:len(df_X_scaled)],
    'faulty': y_resampled,  # y_resampled è ora 1D
    'trq_margin': df_train['trq_margin'][:len(df_X_scaled)]
})

# 6. Salvare i nuovi DataFrame nei file CSV
df_X_scaled.to_csv("Dataset/Train/X_train_resampled.csv", index=False)
y_train_final.to_csv("Dataset/Train/y_train_resampled.csv", index=False)

print("I file sono stati salvati correttamente come X_train_resampled.csv e y_train_resampled.csv.")

I file sono stati salvati correttamente come X_train_resampled.csv e y_train_resampled.csv.


In [2]:
df_X_scaled.describe()

,trq_measured,oat,mgt,pa,ias,np,ng
count,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05,4.000000e+05
mean,-1.611795e-15,-2.578204e-16,2.278533e-16,1.392664e-16,-2.952305e-16,-1.413341e-15,-3.161915e-17
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,-3.667485e+00,-3.761772e+00,-3.061163e+00,-1.312339e+00,-2.203377e+00,-3.174565e+00,-1.505902e+00
25%,-6.714813e-01,-6.379957e-01,-7.650272e-01,-6.722853e-01,-8.183705e-01,-5.064390e-01,-9.102593e-01
50%,3.465311e-02,1.647758e-01,-7.014400e-02,-3.332354e-01,2.269506e-01,5.816040e-01,-1.816337e-01
75%,7.633213e-01,7.342750e-01,7.707653e-01,3.278751e-01,8.958160e-01,6.468636e-01,1.223179e+00
max,2.925078e+00,2.955990e+00,3.152503e+00,4.410292e+00,1.568183e+00,1.038936e+00,1.590251e+00


In [3]:
y_train_final.describe()

,id,faulty,trq_margin
count,400000.000000,400000.000000,400000.000000
mean,199999.500000,0.499727,-1.153898
std,115470.198175,0.500001,14.071843
min,0.000000,0.000000,-74.747111
25%,99999.750000,0.000000,-6.595603
50%,199999.500000,0.000000,1.689704
75%,299999.250000,1.000000,7.788254
max,399999.000000,1.000000,33.597754
